전처리

In [2]:
import networkx as nx
import random
import pandas as pd
import geopandas as gpd
import folium
import pandas as pd
from shapely.geometry import MultiLineString, LineString
import ortools
from shapely.ops import nearest_points
from concurrent.futures import ProcessPoolExecutor
import multiprocessing
import numpy as np

In [ ]:

# LINK 파일 불러오기
link_df=gpd.read_file(".\[2024-03-25]NODELINKDATA\MOCT_LINK.shp",encoding='euc-kr')
link_df['geometry'] = link_df['geometry'].to_crs('EPSG:4326') # 위도경도 좌표계 변환

# 새로운 DBF 파일 읽기
new_df = gpd.read_file("2022-TM-GR-MR-LLV2 도로망(2021년 기준_세계)\\2022-TM-GR-MR-LLV2 도로망(2021년 기준)\\02. 링크\\ad0022_2021_GR.dbf")
new_df['geometry'] = new_df['geometry'].to_crs('EPSG:4326') # 위도경도 좌표계 변환

link_columns_to_drop = ['UPDATEDATE', 'REMARK', 'HIST_TYPE', 'HISTREMARK','REST_VEH', 'REST_W','REST_H','REMARK','MULTI_LINK','CONNECT','C-ITS']
link_df = link_df.drop(columns=link_columns_to_drop, errors='ignore')

tempL=link_df['LINK_ID'].apply(lambda x:int(x[:3]))
slink_df=link_df[(tempL>=100)&(tempL<=124)] ### 서울 코드 100~124

# 국토교통부_전국 노드링크별 평균택시속도_20231231 (1) 파일 불러오기
taxi_speed_df = pd.read_csv(".\국토교통부_전국 노드링크별 평균택시속도_20231231 (1).csv")

# '링크아이디' 열 이름을 'LINK_ID'로 변경
taxi_speed_df = taxi_speed_df.rename(columns={'링크아이디': 'LINK_ID'})

# 데이터 타입을 일치시키기 위해 LINK_ID를 문자열로 변환
slink_df['LINK_ID'] = slink_df['LINK_ID'].astype(str)
taxi_speed_df['LINK_ID'] = taxi_speed_df['LINK_ID'].astype(str)

# 필요한 열만 남기기: LINK_ID와 평균속도
taxi_speed_df = taxi_speed_df[['LINK_ID', '평균속도']]

# slink_df와 taxi_speed_df를 LINK_ID를 기준으로 병합
slink_df = slink_df.merge(taxi_speed_df, on='LINK_ID', how='left')

# 병합 결과 확인
slink_df.head()


# NODE 파일 불러오기

node_df = gpd.read_file(".\[2024-03-25]NODELINKDATA\MOCT_NODE.shp", encoding='euc-kr')
node_df['geometry'] = node_df['geometry'].to_crs('EPSG:4326')  # 위도경도 좌표계 변환

node_columns_to_drop = ['UPDATEDATE', 'REMARK', 'HIST_TYPE', 'HISTREMARK']
node_df = node_df.drop(columns=node_columns_to_drop, errors='ignore')

tempN=node_df['NODE_ID'].apply(lambda x:int(x[:3]))
snode_df=node_df[(tempN>=100)&(tempN<=124)] ### 서울 코드 100~124


Link 파일 전처리

In [ ]:
#교통망 GIS DB와 ITS 국가교통정보센터에서 쓰는 LINK_ID 일치화
'''
def find_nearest_geometry(target_geom, geometries):
    if target_geom is None:
        return None
    nearest_geom = min(geometries, key=lambda geom: target_geom.distance(geom) if geom is not None else float('inf'))
    return nearest_geom

# 완전히 일치하는 geometry 찾기
exact_match = gpd.sjoin(slink_df, new_df[['geometry', 'TL_DENSITY']], how='inner', predicate='within')
exact_match = exact_match[exact_match.apply(lambda row: row['geometry'].equals(new_df.loc[row['index_right'], 'geometry']), axis=1)]

# 일치하지 않는 geometry에 대해 가장 가까운 geometry 찾기
unmatched = slink_df[~slink_df['LINK_ID'].isin(exact_match['LINK_ID'])]
nearest_match = gpd.sjoin_nearest(unmatched, new_df[['geometry', 'TL_DENSITY']], how='left', max_distance=1000)  # max_distance 추가

# 중복 제거 (LINK_ID 기준으로 첫 번째 매치만 유지)
nearest_match = nearest_match.groupby('LINK_ID').first().reset_index()

# 결과 합치기
merged_df = pd.concat([exact_match, nearest_match])

# 중복 열 제거 및 인덱스 재설정
merged_df = merged_df.drop(columns=['index_right'], errors='ignore').reset_index(drop=True)

# NaN 값 처리 (선택적)
merged_df = merged_df.dropna(subset=['geometry'])

# 결과를 slink_df에 할당
slink_df = merged_df

'''

In [ ]:
'''
#빈 평균속도 값 NODE 꼬리물면서 찾기

def fill_missing_speed_recursive(df, link_id, visited=None, depth=0):
    if visited is None:
        visited = set()
    
    if link_id in visited or depth > 10:  # 깊이 제한 추가
        return None
    
    visited.add(link_id)
    row = df[df['LINK_ID'] == link_id].iloc[0]
    
    if pd.notnull(row['평균속도']):
        return row['평균속도']
    
    # F_NODE와 일치하는 T_NODE를 가진 행들 찾기
    matching_rows_f = df[(df['T_NODE'] == row['F_NODE']) & (df['LINK_ID'] != link_id)]
    
    for _, matching_row in matching_rows_f.iterrows():
        speed = fill_missing_speed_recursive(df, matching_row['LINK_ID'], visited, depth + 1)
        if speed is not None:
            return speed
    
    # T_NODE와 일치하는 F_NODE를 가진 행들 찾기
    matching_rows_t = df[(df['F_NODE'] == row['T_NODE']) & (df['LINK_ID'] != link_id)]
    
    for _, matching_row in matching_rows_t.iterrows():
        speed = fill_missing_speed_recursive(df, matching_row['LINK_ID'], visited, depth + 1)
        if speed is not None:
            return speed
    
    return None

def fill_missing_speed(df):
    for index, row in df[df['평균속도'].isnull()].iterrows():
        speed = fill_missing_speed_recursive(df, row['LINK_ID'])
        if speed is not None:
            df.at[index, '평균속도'] = speed
    return df

# slink_df에 함수 적용
slink_df = fill_missing_speed(slink_df)

# 결과 확인
print(slink_df[['LINK_ID', 'F_NODE', 'T_NODE', 'ROAD_RANK', '평균속도']])

# 남아있는 null 값 확인
null_count = slink_df['평균속도'].isnull().sum()
print(f"남아있는 null 값의 개수: {null_count}")

# 남아있는 null 값에 대해 ROAD_RANK별 평균 속도 적용
if null_count > 0:
    mean_speed_by_rank = slink_df.groupby('ROAD_RANK')['평균속도'].mean()
    
    for index, row in slink_df[slink_df['평균속도'].isnull()].iterrows():
        slink_df.at[index, '평균속도'] = mean_speed_by_rank[row['ROAD_RANK']]

    # 최종 결과 확인
    print("ROAD_RANK별 평균 속도 적용 후:")
    print(slink_df[['LINK_ID', 'F_NODE', 'T_NODE', 'ROAD_RANK', '평균속도']])
    
    # 최종 null 값 확인
    final_null_count = slink_df['평균속도'].isnull().sum()
    print(f"최종 남아있는 null 값의 개수: {final_null_count}")
    '''
    

In [ ]:
#파라미터 적용
'''
#vdr_parameter.CSV 파일 읽기
vdr_params = pd.read_csv('vdr_parameter.CSV')

# 숫자 컬럼의 쉼표 제거 및 숫자형으로 변환
numeric_columns = ['C_L', 'C_S', 'C_U', 'ROAD_RANK_L', 'ROAD_RANK_U', 'LANES_L', 'LANES_U', 'TL_DENSITY_L', 'TL_DENSITY_U']
for col in numeric_columns:
    vdr_params[col] = pd.to_numeric(vdr_params[col].astype(str).str.replace(',', ''), errors='coerce')

# link_df의 관련 열들을 숫자형으로 변환
slink_df['ROAD_RANK'] = pd.to_numeric(slink_df['ROAD_RANK'], errors='coerce')
slink_df['LANES'] = pd.to_numeric(slink_df['LANES'], errors='coerce')
slink_df['TL_DENSITY'] = pd.to_numeric(slink_df['TL_DENSITY'], errors='coerce')

# 조건에 따라 VDF 값을 할당하는 함수
def assign_vdf(row, vdr_params):
    for _, vdr_row in vdr_params.iterrows():
        road_rank_condition = vdr_row['ROAD_RANK_L'] < row['ROAD_RANK'] <= vdr_row['ROAD_RANK_U']
        lanes_condition = vdr_row['LANES_L'] < row['LANES'] <= vdr_row['LANES_U']
        
        if row['ROAD_RANK'] in [101, 102]:
            # ROAD_RANK가 101 또는 102인 경우 TL_DENSITY 조건을 무시
            if road_rank_condition and lanes_condition:
                return pd.Series({
                    'VDF': vdr_row['VDF'],
                    'BPR_A': vdr_row['BPR_A'],
                    'BPR_B': vdr_row['BPR_B'],
                    'V_L': vdr_row['V_L'],
                    'V_S': vdr_row['V_S'],
                    'V_U': vdr_row['V_U'],
                    'C_L': vdr_row['C_L'],
                    'C_S': vdr_row['C_S'],
                    'C_U': vdr_row['C_U']
                })
        else:
            # TL_DENSITY가 0인 경우 특별 처리
            if row['TL_DENSITY'] == 0:
                tl_density_condition = vdr_row['TL_DENSITY_L'] == 0 and vdr_row['TL_DENSITY_U'] == 0.3
            elif pd.isna(row.get('TL_DENSITY')):
                tl_density_condition = True  # TL_DENSITY가 NaN이면 조건을 무시
            else:
                tl_density_condition = vdr_row['TL_DENSITY_L'] < row['TL_DENSITY'] <= vdr_row['TL_DENSITY_U']
            
            if road_rank_condition and lanes_condition and tl_density_condition:
                return pd.Series({
                    'VDF': vdr_row['VDF'],
                    'BPR_A': vdr_row['BPR_A'],
                    'BPR_B': vdr_row['BPR_B'],
                    'V_L': vdr_row['V_L'],
                    'V_S': vdr_row['V_S'],
                    'V_U': vdr_row['V_U'],
                    'C_L': vdr_row['C_L'],
                    'C_S': vdr_row['C_S'],
                    'C_U': vdr_row['C_U']
                })
    
    # 조건에 맞는 값이 없을 경우 기본값 반환
    return pd.Series({
        'VDF': None,
        'BPR_A': None,
        'BPR_B': None,
        'V_L': None,
        'V_S': None,
        'V_U': None,
        'C_L': None,
        'C_S': None,
        'C_U': None
    })

# slink_df에 VDF 값 할당
new_columns = slink_df.apply(lambda row: assign_vdf(row, vdr_params), axis=1)
slink_df = pd.concat([slink_df, new_columns], axis=1)

display(slink_df)
'''


In [ ]:
'''
# slink_df를 CSV 파일로 저장
slink_df.to_csv("slink_df_output.csv", index=False, encoding='utf-8-sig')
'''

In [ ]:
#LinK 시각화 함수

def linechart(df):
    m = folium.Map(location=[37.5665, 126.9780], zoom_start=15)
    for _, row in df.iterrows():
        if isinstance(row['geometry'], MultiLineString) and len(row['geometry'].geoms) > 1:
            for line_string in row['geometry'].geoms:
                coordinates = [(lat, lon) for lon, lat in line_string.coords]
                folium.PolyLine(locations=coordinates, color='blue').add_to(m)
        else:
            coordinates = [(lat, lon) for lon, lat in zip(*row['geometry'].xy)]
            folium.PolyLine(locations=coordinates, color='blue').add_to(m)
    return m

linechart(slink_df)

In [ ]:
'''
display(snode_df)
'''

In [ ]:
'''
# NODE 시각화 함수
def node_chart(df):
    m = folium.Map(location=[37.5665, 126.9780], zoom_start=11)
    for _, row in df.iterrows():
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=2,
            popup=row['NODE_ID'],
            color='red',
            fill=True
        ).add_to(m)
    return m

# 서울 지역 NODE 시각화
seoul_node_map = node_chart(snode_df)
display(seoul_node_map)
'''

In [ ]:
"""
# 노드와 링크를 겹쳐서 시각화
def visualize_nodes_and_links(node_df, link_df):
    m = folium.Map(location=[37.5665, 126.9780], zoom_start=15)
    
    # Add links to the map
    for _, row in link_df.iterrows():
        if isinstance(row['geometry'], MultiLineString) and len(row['geometry'].geoms) > 1:
            for line_string in row['geometry'].geoms:
                coordinates = [(lat, lon) for lon, lat in line_string.coords]
                folium.PolyLine(locations=coordinates, color='blue').add_to(m)
        else:
            coordinates = [(lat, lon) for lon, lat in zip(*row['geometry'].xy)]
            folium.PolyLine(locations=coordinates, color='blue').add_to(m)
    
    # Add nodes to the map
    for _, row in node_df.iterrows():
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=2,
            popup=row['NODE_ID'],
            color='red',
            fill=True
        ).add_to(m)
    
    return m

# 시각화
combined_map = visualize_nodes_and_links(snode_df, slink_df)
display(combined_map)
"""

BPR 함수를 이용해서 교통량 추정

In [ ]:
'''
def calculate_volume(row):
    C = row['C_S']
    alpha = row['BPR_A']
    beta = row['BPR_B']
    t = row['LENGTH'] / row['평균속도']
    t_0 = row['LENGTH'] / row['V_U']
    
    # BPR 함수를 이용하여 V 계산
    if t <= t_0:
        return 0  # 현재 통행시간이 자유 통행시간보다 작거나 같으면 교통량은 0
    
    V = C * ((t / t_0 - 1) / alpha) ** (1 / beta)
    return V

# slink_df에 새로운 열 'Calculated_Volume' 추가
slink_df['Calculated_Volume'] = slink_df.apply(calculate_volume, axis=1)

# 결과 확인
print(slink_df[['LINK_ID', 'LENGTH', '평균속도', 'V_S', 'C_S', 'BPR_A', 'BPR_B', 'Calculated_Volume']])
'''

링크에 구명 배정

In [ ]:
'''
slink_df.to_csv("slink_df_output_with_q.csv", index=False, encoding='utf-8-sig')
'''

In [ ]:
'''
# 필요한 라이브러리 import
import pandas as pd
import geopandas as gpd

# 도로명코드_전체 파일 읽기 
road_code_df = pd.read_csv("도로명코드_전체.csv", encoding='euc-kr')

# slink_df 읽기 (이미 있다고 가정)
# slink_df = gpd.read_file("path_to_slink_file.shp")

# 도로명과 시군구명 매칭 함수
def match_road_name(row, road_code_df):
    matched = road_code_df[road_code_df['도로명'] == row['ROAD_NAME']]
    if not matched.empty:
        return matched.iloc[0]['시군구명']
    return None

# 연결된 링크를 통해 시군구명 찾기 함수
def find_sigungu_recursive(df, link_id, visited=None, depth=0):
    if visited is None:
        visited = set()
    
    if link_id in visited or depth > 10:  # 깊이 제한
        return None
    
    visited.add(link_id)
    row = df[df['LINK_ID'] == link_id].iloc[0]
    
    if pd.notnull(row['시군구명']):
        return row['시군구명']
    
    # F_NODE와 일치하는 T_NODE를 가진 행들 찾기
    matching_rows_f = df[(df['T_NODE'] == row['F_NODE']) & (df['LINK_ID'] != link_id)]
    
    for _, matching_row in matching_rows_f.iterrows():
        sigungu = find_sigungu_recursive(df, matching_row['LINK_ID'], visited, depth + 1)
        if sigungu is not None:
            return sigungu
    
    # T_NODE와 일치하는 F_NODE를 가진 행들 찾기
    matching_rows_t = df[(df['F_NODE'] == row['T_NODE']) & (df['LINK_ID'] != link_id)]
    
    for _, matching_row in matching_rows_t.iterrows():
        sigungu = find_sigungu_recursive(df, matching_row['LINK_ID'], visited, depth + 1)
        if sigungu is not None:
            return sigungu
    
    return None

# 도로명 매칭 적용
slink_df['시군구명'] = slink_df.apply(lambda row: match_road_name(row, road_code_df), axis=1)

# 연결된 링크를 통해 시군구명 찾기
for index, row in slink_df[slink_df['시군구명'].isnull()].iterrows():
    sigungu = find_sigungu_recursive(slink_df, row['LINK_ID'])
    if sigungu is not None:
        slink_df.at[index, '시군구명'] = sigungu

# 결과 확인
print(slink_df[['LINK_ID', 'ROAD_NAME', '시군구명']])

# 남아있는 null 값 확인
null_count = slink_df['시군구명'].isnull().sum()
print(f"시군구명이 없는 링크 수: {null_count}")
slink_df.to_csv("slink_df_output_with_q,s.csv", index=False, encoding='utf-8-sig')
'''

In [ ]:
"""# Slink_df_output.csv 파일 읽기
slink_df = pd.read_csv('Slink_df_output_with_q,s.csv')

# 만약 geometry 열이 있다면, 이를 GeoSeries로 변환
if 'geometry' in slink_df.columns:
    slink_df['geometry'] = gpd.GeoSeries.from_wkt(slink_df['geometry'])
    slink_df = gpd.GeoDataFrame(slink_df, geometry='geometry')"""

In [ ]:
"""import numpy as np
from scipy.optimize import fsolve

# 금천구 링크만 선택
geumcheon_links = slink_df[slink_df['시군구명'] == '금천구']

def bpr_inverse(speed, row):
    t = row['LENGTH'] / speed
    t_0 = row['LENGTH'] / row['V_U']
    C = row['C_S']
    V = row['Calculated_Volume'] * 0.67  # 67% of the current volume
    alpha = row['BPR_A']
    beta = row['BPR_B']
    
    return t / t_0 - (1 + alpha * (V / C) ** beta)

def calculate_new_speed(row):
    initial_guess = row['평균속도']
    new_speed = fsolve(bpr_inverse, initial_guess, args=(row,))[0]
    return new_speed

# 새로운 속도 계산
geumcheon_links['New_Speed'] = geumcheon_links.apply(calculate_new_speed, axis=1)

# 결과 출력
print(geumcheon_links[['LINK_ID', '평균속도', 'New_Speed', 'Calculated_Volume']])

# 평균 속도 변화 계산
avg_speed_change = (geumcheon_links['New_Speed'].mean() - geumcheon_links['평균속도'].mean()) / geumcheon_links['평균속도'].mean() * 100

print(f"\n평균 속도 변화: {avg_speed_change:.2f}%")"""

In [ ]:
"""import pandas as pd
import geopandas as gpd
import networkx as nx
from shapely.geometry import LineString
import numpy as np

# 1. 데이터 로드 및 준비
geumcheon_links = pd.read_csv('geumcheon_links.csv')
gdf = gpd.GeoDataFrame(geumcheon_links, geometry=gpd.GeoSeries.from_wkt(geumcheon_links['geometry']))

def distribute_flow_ue(G, paths, total_flow, max_iterations=100, convergence_threshold=0.01):

    n_paths = len(paths)
    flows = np.full(n_paths, total_flow / n_paths)  # 초기에는 균등하게 분배
    
    for _ in range(max_iterations):
        old_flows = flows.copy()
        times = np.array([calculate_travel_time(G, path, flow) for path, flow in zip(paths, flows)])
        
        # 최소 통행 시간을 가진 경로로 통행량을 이동
        min_time = np.min(times)
        flow_to_move = total_flow * 0.1  # 한 번에 이동할 통행량의 비율
        
        for i in range(n_paths):
            if times[i] > min_time:
                flows[i] = max(0, flows[i] - flow_to_move)
            else:
                flows[i] += flow_to_move
        
        # 총 통행량 유지
        flows = flows * (total_flow / np.sum(flows))
        
        # 수렴 확인
        if np.max(np.abs(flows - old_flows)) < convergence_threshold:
            break
    
    return flows

# 2. 네트워크 그래프 생성
G = nx.DiGraph()

for idx, row in gdf.iterrows():
    G.add_edge(row['F_NODE'], row['T_NODE'], 
               length=row['LENGTH'], 
               speed=row['New_Speed'],
               link_id=row['LINK_ID'],
               capacity=row['C_U'],
               bpr_a=row['BPR_A'],
               bpr_b=row['BPR_B'])

def calculate_travel_time(G, path, additional_flow=0):
    travel_time = 0
    for i in range(len(path) - 1):
        if G.has_edge(path[i], path[i+1]):
            edge = G[path[i]][path[i+1]]
            link_data = gdf[gdf['LINK_ID'] == edge['link_id']].iloc[0]
            length = edge['length']
            speed = edge['speed']
            capacity = edge['capacity']
            volume = link_data['Calculated_Volume'] + additional_flow
            bpr_a = edge['bpr_a']
            bpr_b = edge['bpr_b']
            
            # BPR 함수
            travel_time += length / speed * (1 + bpr_a * (volume / capacity) ** bpr_b)
        else:
            # 엣지가 존재하지 않으면 큰 값 반환
            return float('inf')
    
    return travel_time

def find_alternative_paths(G, start, end, k=2):
    try:
        paths = list(nx.shortest_simple_paths(G, start, end, weight='length'))
        return [path for path in paths[:k] if all(G.has_edge(path[i], path[i+1]) for i in range(len(path)-1))]
    except nx.NetworkXNoPath:
        return []

# 3. 모든 링크에 대해 반복
braess_paradox_links = []

for idx, link in gdf.iterrows():
    start_node = link['F_NODE']
    end_node = link['T_NODE']
    
    # 현재 링크의 초기 통행 시간 계산
    initial_time_current = calculate_travel_time(G, [start_node, end_node])
    
    # 대체 경로 찾기
    alternative_paths = find_alternative_paths(G, start_node, end_node)
    
    if len(alternative_paths) < 1:
        print(f"링크 {link['LINK_ID']}에 대한 대체 경로를 찾을 수 없습니다. 건너뜁니다.")
        continue
    
    # 대체 경로의 초기 통행 시간 계산
    additional_flow = link['Calculated_Volume']
    new_flows = distribute_flow_ue(G, alternative_paths, additional_flow)
    new_times = [calculate_travel_time(G, path, flow) for path, flow in zip(alternative_paths, new_flows)]
    
    # 링크 제거
    G.remove_edge(start_node, end_node)
    
    # 교통량 재분배
    additional_flow = link['Calculated_Volume'] / len(alternative_paths)
    
    # 대체 경로의 새로운 통행 시간 계산
    new_times = [calculate_travel_time(G, path, additional_flow) for path in alternative_paths]
    
    # Braess의 역설 확인
    if all(new_time < initial_time for new_time, initial_time in zip(new_times, initial_times)):
        print(f"링크 {link['LINK_ID']}에서 Braess의 역설이 관찰되었습니다.")
        braess_paradox_links.append(link['LINK_ID'])
    
    # 그래프에 링크 다시 추가
    G.add_edge(start_node, end_node, 
               length=link['LENGTH'], 
               speed=link['New_Speed'],
               link_id=link['LINK_ID'],
               capacity=link['C_U'],
               bpr_a=link['BPR_A'],
               bpr_b=link['BPR_B'])

print(f"Braess의 역설을 보이는 링크 수: {len(braess_paradox_links)}")
print("Braess의 역설을 보이는 링크:", braess_paradox_links)"""

In [ ]:
"""
import pandas as pd
import geopandas as gpd
import networkx as nx
from shapely.geometry import LineString
import numpy as np

# 결과를 저장할 리스트 생성
results = []

# 1. 데이터 로드 및 준비
geumcheon_links = pd.read_csv('geumcheon_links.csv')
gdf = gpd.GeoDataFrame(geumcheon_links, geometry=gpd.GeoSeries.from_wkt(geumcheon_links['geometry']))

def distribute_flow_ue(G, paths, total_flow, max_iterations=100, convergence_threshold=0.01):

    n_paths = len(paths)
    flows = np.full(n_paths, total_flow / n_paths)  # 초기에는 균등하게 분배

    for _ in range(max_iterations):
        old_flows = flows.copy()
        times = np.array([calculate_travel_time(G, path, flow) for path, flow in zip(paths, flows)])

        # 최소 통행 시간을 가진 경로로 통행량을 이동
        min_time = np.min(times)
        flow_to_move = total_flow * 0.1  # 한 번에 이동할 통행량의 비율

        for i in range(n_paths):
            if times[i] > min_time:
                flows[i] = max(0, flows[i] - flow_to_move)
            else:
                flows[i] += flow_to_move

        # 총 통행량 유지
        flows = flows * (total_flow / np.sum(flows))

        # 수렴 확인
        if np.max(np.abs(flows - old_flows)) < convergence_threshold:
            break

    return flows

# 2. 네트워크 그래프 생성
G = nx.DiGraph()

for idx, row in gdf.iterrows():
    G.add_edge(row['F_NODE'], row['T_NODE'],
               length=row['LENGTH'],
               speed=row['New_Speed'],
               link_id=row['LINK_ID'],
               capacity=row['C_U'],
               bpr_a=row['BPR_A'],
               bpr_b=row['BPR_B'])

def calculate_travel_time(G, path, additional_flow=0):
    travel_time = 0
    for i in range(len(path) - 1):
        if G.has_edge(path[i], path[i+1]):
            edge = G[path[i]][path[i+1]]
            link_data = gdf[gdf['LINK_ID'] == edge['link_id']].iloc[0]
            length = edge['length']
            speed = edge['speed']
            capacity = edge['capacity']
            volume = link_data['Calculated_Volume'] + additional_flow
            bpr_a = edge['bpr_a']
            bpr_b = edge['bpr_b']

            # BPR 함수
            travel_time += length / speed * (1 + bpr_a * (volume / capacity) ** bpr_b)
        else:
            # 엣지가 존재하지 않으면 큰 값 반환
            return float('inf')

    return travel_time


def find_alternative_paths(G, start, end, k=4):  # k= 대채경로의 수 
    try:
        paths = list(nx.shortest_simple_paths(G, start, end, weight='length'))
        return [path for path in paths[:k] if all(G.has_edge(path[i], path[i+1]) for i in range(len(path)-1))]
    except nx.NetworkXNoPath:
        return []

braess_paradox_links = []

for idx, link in gdf.iterrows():
    start_node = link['F_NODE']
    end_node = link['T_NODE']

    # 현재 링크의 초기 통행 시간 계산
    initial_time_current = calculate_travel_time(G, [start_node, end_node])

    # 대체 경로 찾기
    alternative_paths = find_alternative_paths(G, start_node, end_node)  # k=4가 기본값으로 설정됨

    if len(alternative_paths) < 1:
        print(f"링크 {link['LINK_ID']}에 대한 대체 경로를 찾을 수 없습니다. 건너뜁니다.")
        continue

    # 대체 경로의 초기 통행 시간 계산
    additional_flow = link['Calculated_Volume']
    initial_times = [calculate_travel_time(G, path, 0) for path in alternative_paths]

    new_flows = distribute_flow_ue(G, alternative_paths, additional_flow)
    new_times = [calculate_travel_time(G, path, flow) for path, flow in zip(alternative_paths, new_flows)]

    # 링크 제거
    G.remove_edge(start_node, end_node)

    # 교통량 재분배
    additional_flow = link['Calculated_Volume'] / len(alternative_paths)

    # 대체 경로의 새로운 통행 시간 계산
    new_times = [calculate_travel_time(G, path, additional_flow) for path in alternative_paths]

    # Braess의 역설 확인
    is_braess = all(new_time < initial_time for new_time, initial_time in zip(new_times, initial_times))
    if is_braess:
        print(f"링크 {link['LINK_ID']}에서 Braess의 역설이 관찰되었습니다.")
        braess_paradox_links.append(link['LINK_ID'])

    # 결과 저장
    for i, (path, initial_time, new_time) in enumerate(zip(alternative_paths, initial_times, new_times)):
        results.append({
            'Link_ID': link['LINK_ID'],
            'Alternative_Path': ' -> '.join(map(str, path)),
            'Initial_Time': initial_time,
            'New_Time': new_time,
            'Time_Difference': new_time - initial_time,
            'Is_Braess': is_braess
        })

    # 그래프에 링크 다시 추가
    G.add_edge(start_node, end_node,
               length=link['LENGTH'],
               speed=link['New_Speed'],
               link_id=link['LINK_ID'],
               capacity=link['C_U'],
               bpr_a=link['BPR_A'],
               bpr_b=link['BPR_B'])

print(f"Braess의 역설을 보이는 링크 수: {len(braess_paradox_links)}")
print("Braess의 역설을 보이는 링크:", braess_paradox_links)

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

# CSV 파일로 저장
results_df.to_csv('braess_paradox_results.csv', index=False, encoding='utf-8-sig')
print("결과가 'braess_paradox_results.csv' 파일로 저장되었습니다.")"""

In [ ]:
import folium

# 표시할 LINK_ID 목록 정의
link_ids_to_display = ['1170017402']#, '1170040400', '1170040500', '1170005202', '1170017402']

# slink_df에서 해당 LINK_ID를 필터링
filtered_links = slink_df[slink_df['LINK_ID'].isin(link_ids_to_display)]

# 특정 위치를 중심으로 Folium 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=15)

# 필터링된 링크를 지도에 추가
for _, row in filtered_links.iterrows():
    if isinstance(row['geometry'], MultiLineString) and len(row['geometry'].geoms) > 1:
        for line_string in row['geometry'].geoms:
            coordinates = [(lat, lon) for lon, lat in line_string.coords]
            folium.PolyLine(
                locations=coordinates,
                color='red',
                popup=f"LINK_ID: {row['LINK_ID']}"
            ).add_to(m)
    else:
        coordinates = [(lat, lon) for lon, lat in zip(*row['geometry'].xy)]
        folium.PolyLine(
            locations=coordinates,
            color='red',
            popup=f"LINK_ID: {row['LINK_ID']}"
        ).add_to(m)
        
# 지도 저장
m